In [1]:
#import train csv
import pandas as pd
import numpy as np
df = pd.read_csv('kerala_flood_nepal_earthquake_english_only.csv')
#test = pd.read_csv('~/project/nlp-getting-started/test.csv')

In [2]:
#dropping all the things without a numeral in it
#for i in df:
 #   if not(isnumeric(df[i][target]):
 #       df.drop(i)
print(df)      

      Unnamed: 0  index                                              tweet  \
0              0      1  4.1 magnitude earthquake hits western Nepal - ...   
1              1      7  Nepal earthquake: At least six dead as powerfu...   
2              2      8  Anyone know a Nepal fanbase or at least a gc ?...   
3              3     12  @republic Earthquake tremors felt across Delhi...   
4              4     13  @boomerdoc410_ This could be bad. Nepal earthq...   
...          ...    ...                                                ...   
6546        6546   4992  Multi Stain Remover No Acid on Amazon https://...   
6547        6547   4994  Dream Land is the place that gives you space f...   
6548        6548   4996  Will request the Honble governor to call a spe...   
6549        6549   4997  #RelianceFoundation distributed 15 dry ration ...   
6550        6550   4999  I wish all the Kerela people a happy EID MUBAR...   

      target  
0          1  
1          1  
2          0  
3  

In [3]:
df.isnull().sum()

Unnamed: 0    0
index         0
tweet         0
target        0
dtype: int64

In [4]:
# pre processing step
#start with lower casing everything
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
string.punctuation

# print(lemmatizer.lemmatize("Machine", pos='n'))
# # pos: parts of speech tag, verb
# print(lemmatizer.lemmatize("caring", pos='v'))
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

ModuleNotFoundError: No module named 'nltk'

In [ ]:
import nltk
nltk.download()

In [ ]:
stop_words = set(stopwords.words('english')) 
for row in range (0,df.index[-1]):
    str1 = df.loc[row]['tweet']
    str1="".join([i for i in str1 if i not in string.punctuation])  #punctuation removal
    str1 = ''.join([i for i in str1 if not i.isdigit()])
    str1 = str1.lower()#lowercase
    word_tokens = word_tokenize(str1) #tokenize
    filtered_list = [w for w in word_tokens if not w in stop_words]#removal of stop words
    lemmetized_list = [lemmatizer.lemmatize(w) for w in word_tokens]  #lemmetization  
    filtered_sent = ' '.join(lemmetized_list)
    df.at[row, 'text'] = filtered_sent

In [ ]:
df

In [ ]:
X = df['text']
y = df['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape: ', X_test.shape)

In [ ]:
print(X_train)
X_train = X_train.astype(str)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
#p_test.SentimentText=p_test.SentimentText.astype(str)
X_train_counts.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
# otherwise this can be done other than the top 2 steps
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()

# X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
# X_train_tfidf.shape

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

In [ ]:
predictions = text_clf.predict(X_test)
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))
print("accuracy is", metrics.accuracy_score(y_test,predictions))

In [ ]:

examples = [
    'My heart goes out to the victims',  # this is the same sentence tried earlier
    'Its a nice day today',
    'The flood is terrible',
    'The movie was okish.',
    'Hope the families of the people stuck in the earthquake are okay.',
    'this earthquake is poggers man'
]


In [ ]:
text_clf.predict(examples)

In [ ]:
import pickle
filename = '/Major_Project/models/text_clf.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
print('Tensorflow module imported')

In [ ]:
bert_preprocess = hub.KerasLayer("/Major_Project/bert/bert_en_uncased_L-12_H-768_A-12")
bert_encoder = hub.KerasLayer("/Major_Project/bert/bert_en_uncased_L-12_H-768_A-12")
print('encoder and preprocessor ready')

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = bert_preprocess(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))


In [ ]:
# # text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# text_input = tf.keras.layers.Input(shape=(), name="Input", dtype=tf.string)
# preprocessed_text = bert_preprocess(text_input)
# outputs = bert_encoder(preprocessed_text)
# l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
# l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# model = tf.keras.Model(inputs=[text_input], outputs = [l])
# # input = tf.keras.layers.Input(shape=(), name="Input", dtype=tf.string)

In [ ]:
# path = "F:\\Major_Project\\models\\100\\checkpoint.ckpt"

In [ ]:
# model.load_weights(path)

In [ ]:
# model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='bert_checkpoint.ckpt',
#                                                  save_weights_only=True,
#                                                  verbose=1)


In [ ]:
y_train

In [ ]:
model.fit(X_train, y_train, epochs=63,callbacks=[cp_callback])